In [ ]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''import numpy as np
np.random.seed(42)
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D,GlobalAveragePooling1D
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')'''
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

In [ ]:
EMBEDDING_FILE = '/content/drive/My Drive/go/glove.twitter.27B.200d.txt'
train = pd.read_csv('/content/drive/My Drive/Hate Speech/english_dataset.tsv', sep = '\t', encoding="latin-1")
test = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_en_test-2919.tsv', sep = '\t', encoding="latin-1")

In [ ]:
X_train = train["text"]
y_train = train["task_1"]
X_test = test["text"]
y_test = test["task_1"]

In [ ]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english')) 
ix=0
for i in X_train:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_train[ix]=X_train[ix]+(w) 
  ix=ix+1
ix=0
for i in X_test:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_test[ix]= X_test[ix]+(w) 
  ix=ix+1

In [ ]:
#remove punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [ ]:
ix=0
for i in X_train:
  X_train[ix]=clean_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_text(i)
  ix=ix+1

In [ ]:
#clean the numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
ix=0
for i in X_train:
  X_train[ix]=clean_numbers(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_numbers(i)
  ix=ix+1

In [ ]:
#remove contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [ ]:
ix=0
for i in X_train:
  X_train[ix]=replace_contractions(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=replace_contractions(i)
  ix=ix+1

In [ ]:
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [ ]:
ix=0
for i in X_train:
  X_train[ix]=stem_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=stem_text(i)
  ix=ix+1

In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:

max_features = 22107
maxlen = 70
embed_size = 200

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
y_train = y_train.map({'NOT':0, 'HOF': 1})
y_test = y_test.map({'NOT':0, 'HOF': 1})

In [ ]:
y_train=keras.utils.to_categorical(y_train, num_classes=2)
y_test=keras.utils.to_categorical(y_test, num_classes=2)


In [ ]:
y=[]
for i in y_train:
 y.append(np.argmax(i))

In [ ]:
y

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [ ]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
num_filters = 42
filter_sizes = [1,2,3,5]

def get_model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
#    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), 
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    
    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)
        
    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])   
    z = Flatten()(z)
    z = Dropout(0.1)(z)
        
    outp = Dense(2, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [ ]:
# RNN
from keras.layers import Dense, Input, SimpleRNN, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_srr_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    SimpleRNN is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(SimpleRNN(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
srr_model=model_srr_du(embedding_matrix)

In [ ]:
# LSTM
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    LSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
lstm_model=model_lstm_du(embedding_matrix)

In [ ]:
# BiDirectional LSTM
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
ls_model=model_lstm_du(embedding_matrix)

In [ ]:
# GRU
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_gru_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    GRU is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
gr_model=model_gru_du(embedding_matrix)

In [ ]:
batch_size = 256
epochs = 10

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,
                                              random_state=233)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 5559 samples, validate on 293 samples
Epoch 1/10
 - 2s - loss: 0.6678 - accuracy: 0.6041 - val_loss: 0.6613 - val_accuracy: 0.6382
Epoch 2/10
 - 1s - loss: 0.6183 - accuracy: 0.6463 - val_loss: 0.6509 - val_accuracy: 0.6621
Epoch 3/10
 - 1s - loss: 0.5511 - accuracy: 0.7212 - val_loss: 0.6426 - val_accuracy: 0.6553
Epoch 4/10
 - 1s - loss: 0.4488 - accuracy: 0.8059 - val_loss: 0.6386 - val_accuracy: 0.6621
Epoch 5/10
 - 1s - loss: 0.3510 - accuracy: 0.8743 - val_loss: 0.6541 - val_accuracy: 0.6519
Epoch 6/10
 - 1s - loss: 0.2581 - accuracy: 0.9340 - val_loss: 0.6769 - val_accuracy: 0.6758
Epoch 7/10
 - 1s - loss: 0.1813 - accuracy: 0.9658 - val_loss: 0.6993 - val_accuracy: 0.6621
Epoch 8/10
 - 1s - loss: 0.1242 - accuracy: 0.9835 - val_loss: 0.7283 - val_accuracy: 0.6587
Epoch 9/10
 - 1s - loss: 0.0855 - accuracy: 0.9912 - val_loss: 0.7668 - val_accuracy: 0.6485
Epoch 10/10
 - 1s - loss: 0.0627 - accuracy: 0.9923 - val_loss: 0.8072 - val_accuracy: 0.6519


In [ ]:
srr_hist = srr_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 5559 samples, validate on 293 samples
Epoch 1/10
 - 4s - loss: 0.6792 - accuracy: 0.6001 - val_loss: 0.6650 - val_accuracy: 0.6246
Epoch 2/10
 - 3s - loss: 0.6534 - accuracy: 0.6132 - val_loss: 0.6551 - val_accuracy: 0.6246
Epoch 3/10
 - 3s - loss: 0.6412 - accuracy: 0.6206 - val_loss: 0.6566 - val_accuracy: 0.6246
Epoch 4/10
 - 3s - loss: 0.5864 - accuracy: 0.6868 - val_loss: 0.6497 - val_accuracy: 0.6621
Epoch 5/10
 - 3s - loss: 0.4738 - accuracy: 0.7816 - val_loss: 0.6994 - val_accuracy: 0.5939
Epoch 6/10
 - 3s - loss: 0.3236 - accuracy: 0.8813 - val_loss: 0.7897 - val_accuracy: 0.5461
Epoch 7/10
 - 3s - loss: 0.1969 - accuracy: 0.9432 - val_loss: 0.8285 - val_accuracy: 0.6314
Epoch 8/10
 - 3s - loss: 0.1004 - accuracy: 0.9763 - val_loss: 0.9164 - val_accuracy: 0.6382
Epoch 9/10
 - 3s - loss: 0.0550 - accuracy: 0.9903 - val_loss: 0.9981 - val_accuracy: 0.6416
Epoch 10/10
 - 3s - loss: 0.0378 - accuracy: 0.9932 - val_loss: 1.0652 - val_accuracy: 0.6109


In [ ]:
lstm_hist = lstm_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 5559 samples, validate on 293 samples
Epoch 1/10
 - 6s - loss: 0.6605 - accuracy: 0.6154 - val_loss: 0.6453 - val_accuracy: 0.6246
Epoch 2/10
 - 4s - loss: 0.6198 - accuracy: 0.6573 - val_loss: 0.6239 - val_accuracy: 0.6655
Epoch 3/10
 - 4s - loss: 0.5609 - accuracy: 0.7163 - val_loss: 0.6081 - val_accuracy: 0.6655
Epoch 4/10
 - 4s - loss: 0.4797 - accuracy: 0.7692 - val_loss: 0.6389 - val_accuracy: 0.6826
Epoch 5/10
 - 4s - loss: 0.3482 - accuracy: 0.8507 - val_loss: 0.7684 - val_accuracy: 0.6758
Epoch 6/10
 - 4s - loss: 0.2156 - accuracy: 0.9199 - val_loss: 0.8422 - val_accuracy: 0.6382
Epoch 7/10
 - 4s - loss: 0.1210 - accuracy: 0.9611 - val_loss: 1.1032 - val_accuracy: 0.6416
Epoch 8/10
 - 4s - loss: 0.0685 - accuracy: 0.9793 - val_loss: 1.2846 - val_accuracy: 0.5870
Epoch 9/10
 - 4s - loss: 0.0485 - accuracy: 0.9845 - val_loss: 1.6073 - val_accuracy: 0.6485
Epoch 10/10
 - 4s - loss: 0.0287 - accuracy: 0.9924 - val_loss: 1.6743 - val_accuracy: 0.6382


In [ ]:
ls_hist = ls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 5559 samples, validate on 293 samples
Epoch 1/10
 - 2s - loss: 0.6614 - accuracy: 0.6122 - val_loss: 0.6500 - val_accuracy: 0.6212
Epoch 2/10
 - 1s - loss: 0.6237 - accuracy: 0.6528 - val_loss: 0.6303 - val_accuracy: 0.6314
Epoch 3/10
 - 1s - loss: 0.5664 - accuracy: 0.7095 - val_loss: 0.6440 - val_accuracy: 0.6109
Epoch 4/10
 - 1s - loss: 0.4771 - accuracy: 0.7701 - val_loss: 0.6616 - val_accuracy: 0.6997
Epoch 5/10
 - 1s - loss: 0.3641 - accuracy: 0.8403 - val_loss: 0.7888 - val_accuracy: 0.6075
Epoch 6/10
 - 1s - loss: 0.2194 - accuracy: 0.9167 - val_loss: 0.8517 - val_accuracy: 0.6075
Epoch 7/10
 - 1s - loss: 0.1166 - accuracy: 0.9622 - val_loss: 1.1603 - val_accuracy: 0.6177
Epoch 8/10
 - 1s - loss: 0.0661 - accuracy: 0.9788 - val_loss: 1.5797 - val_accuracy: 0.5802
Epoch 9/10
 - 1s - loss: 0.0415 - accuracy: 0.9874 - val_loss: 1.5037 - val_accuracy: 0.5973
Epoch 10/10
 - 1s - loss: 0.0296 - accuracy: 0.9930 - val_loss: 1.8612 - val_accuracy: 0.6007


In [ ]:
gr_hist = gr_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 5559 samples, validate on 293 samples
Epoch 1/10
 - 6s - loss: 0.6629 - accuracy: 0.6113 - val_loss: 0.6557 - val_accuracy: 0.6246
Epoch 2/10
 - 5s - loss: 0.6282 - accuracy: 0.6427 - val_loss: 0.6434 - val_accuracy: 0.6416
Epoch 3/10
 - 5s - loss: 0.5731 - accuracy: 0.7023 - val_loss: 0.6260 - val_accuracy: 0.6655
Epoch 4/10
 - 5s - loss: 0.4909 - accuracy: 0.7600 - val_loss: 0.6372 - val_accuracy: 0.6826
Epoch 5/10
 - 5s - loss: 0.3779 - accuracy: 0.8412 - val_loss: 0.7113 - val_accuracy: 0.6314
Epoch 6/10
 - 5s - loss: 0.2383 - accuracy: 0.9149 - val_loss: 0.8416 - val_accuracy: 0.5973
Epoch 7/10
 - 5s - loss: 0.1222 - accuracy: 0.9664 - val_loss: 0.9900 - val_accuracy: 0.6143
Epoch 8/10
 - 5s - loss: 0.0561 - accuracy: 0.9872 - val_loss: 1.1427 - val_accuracy: 0.6041
Epoch 9/10
 - 5s - loss: 0.0335 - accuracy: 0.9930 - val_loss: 1.3045 - val_accuracy: 0.6007
Epoch 10/10
 - 5s - loss: 0.0240 - accuracy: 0.9950 - val_loss: 1.3977 - val_accuracy: 0.5973


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(x_test)
y_p=[]
y_t=[]
for i in y_pred:
  y_p.append(np.argmax(i))
for i in y_test:
  y_t.append(np.argmax(i))

In [ ]:
print('Model = CNN\n')
print(classification_report(y_t, y_p))

Model = CNN

              precision    recall  f1-score   support

           0       0.88      0.76      0.82       865
           1       0.49      0.69      0.57       288

    accuracy                           0.74      1153
   macro avg       0.69      0.73      0.70      1153
weighted avg       0.78      0.74      0.76      1153



In [ ]:
y_srr_pred = srr_model.predict(x_test)
y_srr_p=[]
for i in y_srr_pred:
  y_srr_p.append(np.argmax(i))

In [ ]:
print('Model = SimpleRNN\n')
print(classification_report(y_t, y_srr_p))

Model = SimpleRNN

              precision    recall  f1-score   support

           0       0.85      0.68      0.76       865
           1       0.40      0.63      0.49       288

    accuracy                           0.67      1153
   macro avg       0.62      0.66      0.62      1153
weighted avg       0.74      0.67      0.69      1153



In [ ]:
y_lstm_pred = lstm_model.predict(x_test)
y_lstm_p=[]
for i in y_lstm_pred:
  y_lstm_p.append(np.argmax(i))

In [ ]:
print('Model = LSTM\n')
print(classification_report(y_t, y_lstm_p))

Model = LSTM

              precision    recall  f1-score   support

           0       0.85      0.77      0.80       865
           1       0.45      0.58      0.51       288

    accuracy                           0.72      1153
   macro avg       0.65      0.67      0.66      1153
weighted avg       0.75      0.72      0.73      1153



In [ ]:
y_ls_pred = ls_model.predict(x_test)
y_ls_p=[]
for i in y_ls_pred:
  y_ls_p.append(np.argmax(i))


In [ ]:
print('Model = Bidirectonal LSTM\n')
print(classification_report(y_t, y_ls_p))

Model = Bidirectonal LSTM

              precision    recall  f1-score   support

           0       0.84      0.69      0.76       865
           1       0.39      0.61      0.48       288

    accuracy                           0.67      1153
   macro avg       0.62      0.65      0.62      1153
weighted avg       0.73      0.67      0.69      1153



In [ ]:
y_gr_pred = gr_model.predict(x_test)
y_gr_p=[]
for i in y_gr_pred:
  y_gr_p.append(np.argmax(i))

In [ ]:
print('Model = GRU\n')
print(classification_report(y_t, y_gr_p))

Model = GRU

              precision    recall  f1-score   support

           0       0.87      0.73      0.79       865
           1       0.45      0.67      0.54       288

    accuracy                           0.71      1153
   macro avg       0.66      0.70      0.67      1153
weighted avg       0.76      0.71      0.73      1153

